In [78]:
import pyspark

In [79]:
import numpy as np

In [80]:
import pandas as pd

In [81]:
from pyspark.sql import SparkSession

In [82]:
spark= SparkSession.builder.master('local[*]').appName('first_application').getOrCreate()

In [83]:
spark.version

'3.2.1'

In [84]:
flights= spark.read.csv('../Datasets/flights.csv', header=True, inferSchema=True, nullValue='NA')

In [85]:
flights.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

In [86]:
flights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: integer (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: integer (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)



In [87]:
flights.dtypes

[('year', 'int'),
 ('month', 'int'),
 ('day', 'int'),
 ('dep_time', 'int'),
 ('dep_delay', 'int'),
 ('arr_time', 'int'),
 ('arr_delay', 'int'),
 ('carrier', 'string'),
 ('tailnum', 'string'),
 ('flight', 'int'),
 ('origin', 'string'),
 ('dest', 'string'),
 ('air_time', 'int'),
 ('distance', 'int'),
 ('hour', 'int'),
 ('minute', 'int')]

In [88]:
flights.filter('dep_time IS NULL').count()

48

In [89]:
flights.count()

10000

In [90]:
flights= flights.dropna()

In [91]:
flights.count()

9925

In [92]:
from pyspark.sql.functions import round

In [93]:
flights.groupby('origin'

               ).avg('dep_delay').show()

+------+-----------------+
|origin|   avg(dep_delay)|
+------+-----------------+
|   SEA|6.324392787960066|
|   PDX|5.491910392034847|
+------+-----------------+



In [107]:
sortby_air_time= flights.groupby('origin','dest').avg('air_time').sort('avg(air_time)', ascending=False).show(5)

+------+----+------------------+
|origin|dest|     avg(air_time)|
+------+----+------------------+
|   PDX| KOA| 357.6666666666667|
|   SEA| LIH|347.93333333333334|
|   SEA| KOA|           347.075|
|   SEA| HNL|343.50588235294117|
|   PDX| LIH|             341.0|
+------+----+------------------+
only showing top 5 rows



In [110]:
flights.groupby('dest').avg('arr_delay').sort('avg(arr_delay)').show()

+----+-------------------+
|dest|     avg(arr_delay)|
+----+-------------------+
| TPA|              -19.0|
| MSY|-18.555555555555557|
| SAT|-10.444444444444445|
| TUS|               -8.2|
| DCA| -7.720588235294118|
| LIH| -6.405405405405405|
| LMT|               -5.0|
| PSP| -4.595744680851064|
| CLT| -4.589743589743589|
| DTW|-3.6991869918699187|
| KOA| -3.519230769230769|
| OMA|              -3.16|
| CVG|               -3.0|
| LGB|-2.9655172413793105|
| HNL|-2.5774647887323945|
| SIT|-2.3333333333333335|
| ABQ|-2.3157894736842106|
| BUR|-2.0948905109489053|
| SNA|-1.9796954314720812|
| ANC|-1.8507795100222717|
+----+-------------------+
only showing top 20 rows



In [115]:
flights.filter(flights.dest == 'TPA').count()

11

In [116]:
flights.filter(flights.dest == 'MSY').count()

9

In [117]:
flights.filter(flights.dest == 'TUS').count()

50

In [118]:
flights.filter(flights.dest == 'SNA').count()

197

In [119]:
flights.filter(flights.dest == 'ANC').count()

449

In [121]:
flights.filter(flights.dest == 'LAX').count()

611

In [120]:
flights.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 2 rows



In [128]:
# Ich möchte anhand dep_time, carrier, origin und dest, duration prognostizieren und dann evtl delay

In [213]:
flights= flights.withColumn('duration', flights.arr_time- flights.dep_time)

In [130]:
flights.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|     277|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|     465|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
only showing top 2 rows



In [145]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [173]:
si = StringIndexer(inputCols=['carrier', 'origin', 'dest'], outputCols= ['carrier_idx', 'origin_idx', 'dest_idx'])

In [217]:
flights= flights.dropna()

In [218]:
flights_train, flights_test = flights.randomSplit([0.8, 0.2])

In [219]:
flights_train.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
|2014|    1|  1|     600|      -10|     842|       -8|     AS| N786AS|   426|   SEA| LAX|     125|     954|   6|     0|     242|
|2014|    1|  1|     617|        2|     850|      -12|     AS| N579AS|   494|   SEA| SAN|     131|    1050|   6|    17|     233|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
only showing top 2 rows



In [174]:
si =si.fit(flights_train)

In [175]:
flights_train_idx= si.transform(flights_train)

In [176]:
flights_test_idx= si.transform(flights_test)

In [177]:
ohe= OneHotEncoder(inputCols= ['carrier_idx', 'origin_idx', 'dest_idx'], outputCols=['carrier_dum', 'origin_dum', 'dest_dum'])

In [178]:
ohe = ohe.fit(flights_train_idx)

In [179]:
flights_train_dum= ohe.transform(flights_train_idx)

In [180]:
flights_test_dum = ohe.transform(flights_test_idx)

In [181]:
vec = VectorAssembler(inputCols= ['carrier_dum', 'origin_dum', 'dest_dum'], outputCol = 'features')

In [182]:
flights_train_vec= vec.transform(flights_train_dum)

In [183]:
flights_test_vec= vec.transform(flights_test_dum)

In [184]:
from pyspark.ml.regression import LinearRegression

In [185]:
reg= LinearRegression(labelCol='duration')

In [186]:
reg =reg.fit(flights_train_vec)

In [187]:
train_pred= reg.transform(flights_train_vec)

In [188]:
train_pred.select('duration', 'prediction').show()

+--------+------------------+
|duration|        prediction|
+--------+------------------+
|     287|317.94446818479275|
|     233| 245.6221928308347|
|     203| 211.6597502375257|
|     790|-94.57699659642924|
|     577|133.06275720423582|
|     604| 306.5458105217973|
|     731| 207.5453008770902|
|     524|440.43066520650893|
|     587| 447.5092846034605|
|     518| 516.6468035361627|
|     319|229.33342561337042|
|     271|189.76549788495993|
|     139|204.58113084057413|
|     402| 311.9107654575565|
|     591| 42.11436520287259|
|      92| 13.94120711300559|
|     233| 310.8658487878412|
|     715| 295.8134951571699|
|     779|-479.0188066892241|
|     183|103.69732157112048|
+--------+------------------+
only showing top 20 rows



In [189]:
from pyspark.ml.evaluation import RegressionEvaluator

In [190]:
evaluator= RegressionEvaluator(labelCol='duration')

In [191]:
evaluator.evaluate(train_pred)

517.5442071164075

In [193]:
from pyspark.ml.regression import DecisionTreeRegressor

In [194]:
tree= DecisionTreeRegressor(labelCol='duration')

In [195]:
tree= tree.fit(flights_train_vec)

In [196]:
flights_pred_tree= tree.transform(flights_train_vec)

In [197]:
flights_pred_tree_test = tree.transform(flights_test_vec)

In [199]:
evaluator2 = RegressionEvaluator(labelCol = 'duration')

In [200]:
evaluator2.evaluate(flights_pred_tree)

527.6412619835284

In [201]:
evaluator2.evaluate(flights_pred_tree_test)

568.300183717068

In [202]:
from pyspark.ml import Pipeline

In [204]:
flights.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|     277|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|     465|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------+
only showing top 2 rows



In [220]:
indexer2 = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

In [221]:
ohe2 = OneHotEncoder(inputCols=['carrier_idx'], outputCols=['carrier_dum'])

In [224]:
vec2 = VectorAssembler(inputCols=['carrier_dum', 'distance'], outputCol='features')

In [225]:
linreg2 = LinearRegression(labelCol='duration')

In [226]:
pipeline = Pipeline(stages=[indexer2, ohe2, vec2, linreg2])

In [227]:
pipeline_fit = pipeline.fit(flights_train)

In [228]:
pred_pipeline = pipeline_fit.transform(flights_train)

In [229]:
pred_pipeline_test = pipeline_fit.transform(flights_test)

In [230]:
evaluator2 = RegressionEvaluator(labelCol='duration')

In [231]:
evaluator2.evaluate(pred_pipeline)

571.041728495263

In [232]:
evaluator2.evaluate(pred_pipeline_test)

552.0882847398257